***GENERATED CODE FOR longcovidbyage PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Indicator', 'transformation_label': 'String Indexer'}], 'feature': 'Indicator', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'Currently experiencing long COVID, as a percentage of adults who ever had COVID', 'max': 'Ever had COVID', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Indicator'}, {'feature_label': 'Indicator', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Indicator')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Group', 'transformation_label': 'String Indexer'}], 'feature': 'Group', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'By Age', 'max': 'National Estimate', 'missing': '0', 'distinct': '9'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Group'}, {'feature_label': 'Group', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Group')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Alabama', 'max': 'Wyoming', 'missing': '0', 'distinct': '52'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Subgroup', 'transformation_label': 'String Indexer'}], 'feature': 'Subgroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '18 - 29 years', 'max': 'Wyoming', 'missing': '0', 'distinct': '78'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Subgroup'}, {'feature_label': 'Subgroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Subgroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Time Period Label', 'transformation_label': 'String Indexer'}], 'feature': 'Time Period Label', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Jul 23 - Aug 19, 2024', 'max': 'Jun 29 - Jul 11, 2022', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Time Period Label'}, {'feature_label': 'Time Period Label', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Time Period Label')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Time Period Start Date', 'transformation_label': 'String Indexer'}], 'feature': 'Time Period Start Date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '6/1/2022', 'max': '7/23/2024', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Time Period Start Date'}, {'feature_label': 'Time Period Start Date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Time Period Start Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Time Period End Date', 'transformation_label': 'String Indexer'}], 'feature': 'Time Period End Date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '6/13/2022', 'max': '8/19/2024', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Time Period End Date'}, {'feature_label': 'Time Period End Date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Time Period End Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Confidence Interval', 'transformation_label': 'String Indexer'}], 'feature': 'Confidence Interval', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '498', 'mean': '', 'stddev': '', 'min': '10.0 - 11.1', 'max': '9.9 - 15.3', 'missing': '2', 'distinct': '492'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Confidence Interval'}, {'feature_label': 'Confidence Interval', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Confidence Interval')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Quartile range', 'transformation_label': 'String Indexer'}], 'feature': 'Quartile range', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '311', 'mean': '', 'stddev': '', 'min': '11.6-16.9', 'max': 'Estimate not reliable', 'missing': '189', 'distinct': '29'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Quartile range'}, {'feature_label': 'Quartile range', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Quartile range')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run longcovidbyageHooks.ipynb
try:
	#sourcePreExecutionHook()

	postcovidconditions = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Post-COVID_Conditions_20250422.csv', 'filename': 'Post-COVID_Conditions_20250422.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run longcovidbyageHooks.ipynb
try:
	#transformationPreExecutionHook()

	longcovidbyage = TransformationMain.run(postcovidconditions,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Indicator", "transformation_label": "String Indexer"}], "feature": "Indicator", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Currently experiencing long COVID, as a percentage of adults who ever had COVID", "max": "Ever had COVID", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Indicator"}, {"transformationsData": [{"feature_label": "Group", "transformation_label": "String Indexer"}], "feature": "Group", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "By Age", "max": "National Estimate", "missing": "0", "distinct": "9"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Group"}, {"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Alabama", "max": "Wyoming", "missing": "0", "distinct": "52"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"feature_label": "Subgroup", "transformation_label": "String Indexer"}], "feature": "Subgroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "18 - 29 years", "max": "Wyoming", "missing": "0", "distinct": "78"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Subgroup"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Phase", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.51", "stddev": "0.06", "min": "3.5", "max": "4.2", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Phase"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Time Period", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "46.41", "stddev": "2.25", "min": "46", "max": "71", "missing": "0"}, "updatedLabel": "Time Period"}, {"transformationsData": [{"feature_label": "Time Period Label", "transformation_label": "String Indexer"}], "feature": "Time Period Label", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Jul 23 - Aug 19, 2024", "max": "Jun 29 - Jul 11, 2022", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Time Period Label"}, {"transformationsData": [{"feature_label": "Time Period Start Date", "transformation_label": "String Indexer"}], "feature": "Time Period Start Date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "6/1/2022", "max": "7/23/2024", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Time Period Start Date"}, {"transformationsData": [{"feature_label": "Time Period End Date", "transformation_label": "String Indexer"}], "feature": "Time Period End Date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "6/13/2022", "max": "8/19/2024", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Time Period End Date"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Value", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "498", "mean": "23.4", "stddev": "13.2", "min": "2.8", "max": "66.7", "missing": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Value"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LowCI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "498", "mean": "19.76", "stddev": "12.23", "min": "2.0", "max": "62.5", "missing": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LowCI"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "HighCI", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "498", "mean": "27.45", "stddev": "14.41", "min": "3.8", "max": "70.8", "missing": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "HighCI"}, {"transformationsData": [{"feature_label": "Confidence Interval", "transformation_label": "String Indexer"}], "feature": "Confidence Interval", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "498", "mean": "", "stddev": "", "min": "10.0 - 11.1", "max": "9.9 - 15.3", "missing": "2", "distinct": "492"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Confidence Interval"}, {"transformationsData": [{"feature_label": "Quartile range", "transformation_label": "String Indexer"}], "feature": "Quartile range", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "311", "mean": "", "stddev": "", "min": "11.6-16.9", "max": "Estimate not reliable", "missing": "189", "distinct": "29"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Quartile range"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Quartile number", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "311", "mean": "2.43", "stddev": "1.15", "min": "-1", "max": "4", "missing": "189"}, "updatedLabel": "Quartile number"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Suppression Flag", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "2", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "498"}, "updatedLabel": "Suppression Flag"}]}))

	#transformationPostExecutionHook(longcovidbyage)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run longcovidbyageHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(longcovidbyage, ["Phase", "Time Period", "LowCI", "HighCI", "Quartile number", "Suppression Flag", "Indicator_stringindexer", "Group_stringindexer", "State_stringindexer", "Subgroup_stringindexer", "Time Period Label_stringindexer", "Time Period Start Date_stringindexer", "Time Period End Date_stringindexer", "Confidence Interval_stringindexer", "Quartile range_stringindexer"], "Value")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

